In [12]:
%load_ext autoreload
%autoreload 2
import os
import torch

import leafmap


import rasterio
import numpy as np
from doitall import save_poly

from doitall import inference_deadwood 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# load files
data_path = "/mnt/gsdata/projects/deadtrees/drone_campaigns/naturpark_schwarzwald/blackforestnationalparktimeseries/TDOP_mosaic/"
file_name = "TDOP_2023_123_jpg85_btif_mosaic.tif"

file = rasterio.open(data_path + file_name)

In [15]:
file.crs

CRS.from_wkt('PROJCS["ETRS89 / UTM zone 32N",GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4258"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",9],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","25832"]]')

In [34]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import leafmap

# Load the original file
data_path = "/mnt/gsdata/projects/deadtrees/drone_campaigns/naturpark_schwarzwald/blackforestnationalparktimeseries/TDOP_mosaic/"
file_name = "TDOP_2023_123_jpg85_btif_mosaic.tif"

output_path = "/mnt/gsdata/projects/deadtrees/deadwood_segmentation/test/"

with rasterio.open(data_path + file_name) as src:
    # Get the original dimensions
    width = src.width
    height = src.height

    # Calculate the dimensions for the cropped area (1/10 of original size)
    new_width = width // 10
    new_height = height // 10

    # Calculate the starting coordinates to center the crop
    start_x = (width - new_width) // 2
    start_y = (height - new_height) // 2

    # Define the window for cropping
    window = rasterio.windows.Window(start_x, start_y, new_width, new_height)

    # Read data from the window
    data = src.read(window=window)

    # Get the transform for the window
    src_transform = src.window_transform(window)

    # Get the bounds of the window
    bounds = rasterio.windows.bounds(window, src.transform)

    # Define source and destination CRS
    src_crs = src.crs
    dst_crs = 'EPSG:3857'  # Reproject to Web Mercator

    # Set the desired output resolution in meters (e.g., 1 meter per pixel)
    dst_resolution = 1.0  # Adjust as needed

    # Calculate the transform and dimensions for the destination raster
    dst_transform, dst_width, dst_height = calculate_default_transform(
        src_crs, dst_crs,
        width=new_width,
        height=new_height,
        left=bounds[0],
        bottom=bounds[1],
        right=bounds[2],
        top=bounds[3],
        dst_width=None,
        dst_height=None,
        resolution=dst_resolution
    )

    # Update metadata for the destination raster
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': dst_transform,
        'width': dst_width,
        'height': dst_height,
        'nodata': 0  # Ensure NoData value is set appropriately
    })

    # Reproject and write the cropped raster to a new file
    with rasterio.open(output_path + 'temp_crop_reprojected.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            dest = np.zeros((dst_height, dst_width), dtype=data.dtype)

            # Reproject the data
            reproject(
                source=data[i - 1, :, :],
                destination=dest,
                src_transform=src_transform,
                src_crs=src_crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                resampling=Resampling.average  # Use average for downsampling
            )
            # Write the reprojected data to the destination file
            dst.write(dest, indexes=i)

            # Optionally, check if the destination array contains valid data
            valid_data_percentage = np.count_nonzero(dest) / dest.size * 100
            print(f"Band {i}: {valid_data_percentage:.2f}% valid data")


RasterioIOError: Attempt to create new tiff file '/mnt/gsdata/projects/deadtrees/deadwood_segmentation/test/temp_crop_reprojected.tif' failed: No such file or directory

In [ ]:
bounds

In [25]:
m = leafmap.Map()

# Add the reprojected raster
m.add_raster(
    'temp_crop_reprojected.tif',
    layer_name="TDOP 2023 Cropped",
    colormap="terrain",
    nodata=0,
    attribution="TDOP 2023"
)

# Add a basemap
m.add_basemap("SATELLITE")

# Zoom to the extent of the raster layer
# m.zoom_to_bounds(m.layers[-1].bounds)

# Display the map
m


Map(center=[48.5932945, 8.2744345], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…